# 动态内存

- [X] 动态内存与智能指针
- [X] 动态数组
- [ ] 使用标准库：文本查询程序

## 动态内存与智能指针

- 静态内存：保存局部static对象、类static对象、定义在任何函数之外的对象，由编译器自动创建和销毁。栈内存：保存定义在函数之内的非static对象，栈对象仅在定义的程序块运行时才存在
- 除了静态内存和栈内存，每个程序还拥有一个内存池。这部分内存被称作自由空间或堆。程序用堆来存储动态分配的对象
- 要正确地管理动态内存

- 在c++中，动态内存的管理是通过一对运算符来完成的：new，为对象分配空间并放回一个指向该对象的指针，我们可以选择对对象进行初始化
- delete：接受一个动态对象的指针，销毁该对象，并释放与之关联的内存

- 为了防止忘记释放内存和内存泄漏，更容易的使用动态内存，新标准库提供了两种智能指针类型来管理动态对象
- shared_ptr：允许多个指针指向同一个对象;
- unique_ptr: 独占所指的对象
- weak_ptr： 伴随类，弱引用，指向shared_ptr所管理的对象
- 这三种类型都在头文件memory中

### shared_ptr类

```C++
shared_ptr<string> p1;
shared_ptr<list<string>> p2;
```

** make_shared函数 **

- 最安全的分配和使用动态内存的方法是调用一个名make_shared的标准库函数
- 此函数在动态内存中分配一个对象并初始化它，返回指向此对象的shared_ptr

- 当要用make_shared时

```C++
shared_ptr<int> p3 = make_shared<int>(42); //指向一个值为42的int的shared_ptr
shared_ptr<string> p4 = make_shared<string>(10, '9');//指向一个值为“9999999999”的string
shared_ptr<int> p5 = make_shared<int>();
```

- 也可以用auto定义一个对象来保存make_shared的结果

** shared_ptr的拷贝和赋值 **


- 当进行拷贝和赋值时，每个shared_ptr都会记录有多少个其他的shared_ptr指向相同的对象
```C++
auto p = make_shared<int>(42);
auto q(p);
```

```C++
auto r = make_shared<int>(42);
r = q;
```

- 此例中我们分配了一个int，将其指针保存在r中，接下来，我们将新值赋给r，在此情况下，r是唯一指向此int的shared_ptr,再把q赋给r的过程中，此int被自动释放

** shared_ptr自动销毁所管理的对象 **

- 当指向一个对象的最后一个shared_ptr被销毁时，shared_ptr类自动销毁此对象

** shared_ptr还会自动释放相关联的内存 **

定义一个返回类型为shared_ptr的函数

```C++
//factory返回一个shared_ptr,指向一个动态分配的对象
shared_ptr<Foo> factory(T arg){
    reurn make_shared<Foo>(arg);
}
```
> 当factory返回一个shared_ptr,所以我们可以确保它分配的对象会在恰当的时刻被释放

```C++
void use_factory(T arg){
    shared_ptr<Foo> p = factory(arg);
    //使用p
}//p离开作用域，它指向的内存会被自动释放掉
```
>但是如果有其他shared_ptr也指向这块内存，他就不会被释放

```C++
void use_factory(T arg){
    shared_ptr<Foo> p = factory(arg);
    //使用p
    return p; //当我们返回p时，引用计数进行了递增操作
}//p离开作用域，它指向的内存不会被自动释放掉
```

** 使用动态生存器的资源的类 **

使用动态内存的原因：

- 程序不知道自己需要多少对象（容器类）
- 程序不知道所需对象的准确类型
- 程序需要在多个对象间共享数据（本节）

```C++
vector<string> v1;
{
    vector<string> v2 = {"hello", "a", "an"};
    v1 = v2; 
}//v2被销毁，其中的元素也被销毁
//v1有三个元素，是原来v3中元素的拷贝
```

- 我们希望定义一个名为Blob的类，保存一组元素，希望Blob对象的不同拷贝之间共享共同的元素。
- 如果两个对象共享底层的数据 ，当某个对象被销毁时，我们不能单方面的销毁底层数据

```C++
Blob<string> b1;
{
    Blob<string> b2 = {"hello", "a", "an"};
    b1 = b2; 
}//b2被销毁，其中的元素不能销毁
//b1指向由b2创建的元素
```

** 定义strBlob类 **

- 我们不能在一个Blob对象里面直接保存vector,为了保证vector中的元素继续存在，我们将vector保存在动态内存中。

- 为了实现我们所希望的数据共享，我们将为每个strBlob设置一个shared_ptr来管理动态分配的vector。此shared_ptr的成员将记录有多少个StrBlob共享相同的vector，并在vector的最后一个使用者被销毁时释放vector
- 类有一个默认构造函数和一个构造函数

```C++
class StrBlob{
public:
	typedef vector<string>::size_type size_type;
	StrBlob();
	StrBlob(initializer_list<string> il);
	size_type size() const {return data->size();}
	bool empty() const {return data->empty();}
	//add and delete
	void push_back(const string &t) {date->push_back(t);}
	void pop_back();
	//visit
	string &front;
	string &back;

private:
	shared_ptr<vector<string>> data;
	void check(size_type i; const string &msg) const;
};
```

** StrBlob构造函数 **

```C++
StrBlob::StrBlob():data(make_shared<vector<string>>()){}
StrBlob::StrBlob(initializer_list<string> il):data(make_shared<vector<string>>()){}
```

** 元素访问成员函数 **

- 一些操作访问vector的成员函数，必须先检查元素是否存在，定一个一个check的工具函数，检查给定一个索引是否在合法范围内
- 另外一个string参数，传递的是异常处理描述的错误内容

```C++
string& StrBlob::back(){
	check(0, "back on empty StrBlob");
	return data->back();
}

string& StrBlob::front(){
	check(0, "front on empty StrBlob");
	return data->front();
}

string& StrBlob::pop_back(){
	check(0, "pop_back on empty StrBlob");
	data->pop_back();
}
```

** StrBlob的拷贝、赋值和销毁 **


>explicit的作用就是抑制构造函数的隐式转换     
>优点：不会自动的进行类型转换，必须清楚的知道类类型   
>缺点：必须用构造函数显示创建一个对象，不够方便简单   

### 直接管理内存

- new
- delete
> 学得不好的人最好用智能指针来管理内存，否则不要分配动态内存

** 使用new动态分配和初始化对象 **

- 在自由空间分配内存是无名的，因此new无法为其分配的对象命名，而是返回一个指向该对象的指针：
```C++
int *p1 = new int;
```

- 我们可以使用直接初始化方式来初始化一个动态分配的对象，也可以使用传统的构造方式，在新标准下，也可以使用列表初始化（使用花括号）:

```C++
int *pi = new int(1024);
string *ps = new string(10, '9');

vector<int> *pv = new vector<int>{0,1,2,3,44,5,6};
```

- 也可以对动态分配的对象进行值初始化，只需在类型名之后跟一对空括号即可：

```C++
string *ps1 = new string;
string *ps = new string();
string *ps = new int;
int *pi2 = new int();
```

- 只有当括号中仅有单一初始化器的时候才可以使用auto：

```C++
auto p1 = new auto(obj);

auto p2 = new auto{a, b, c};//错误
```

- 根据obj的类型推断初p1的类型

** 动态分配的const对象 **


- 使用new分配const对象是合法的

```C++
const int *pci = new const int(1024);

const string *pcs = new const string;
```

** 内存耗尽 **

- 一旦自由空间被耗尽，new表达式就会失败
- 默认情况下，会抛出一个bad_alloc的异常
- 我们可以改变使用new的方式来阻止它抛出异常

```C++
int *p2 = new int;  //如果分配失败，new抛出bad_alloc
int *p1 = new (nothrow) int;//如果分配失败，返回一个空指针
```

** 释放动态内存 **

- delete p; p必须指向一个动态分配的对象或者是一个空指针
- 为了防止内存耗尽，在动态内存使用完毕后，要归还给系统

** 指针值和delete **

```C++
int i, *pi1 = &i, *pi2 = nullptr;
double *pd = new double(33), *pd2 = pd;
delete i;  //错误：i不是一个指针
delete pi1;  //未定义：pil指向一个局部变量
delete pd;   //正确
delete pd2;   //未定义：pd2指向的内存被释放了
delete pi2;   //正确：释放一个空指针总是没有错误的
```

** 动态对象的生存期直到被释放时为止 **

- 返回指向动态内存的指针（而不是智能指针）的函数给其调用者增加了一个额外负担-调用者必须记得释放内存：

```C++
Foo* factory(T arg){
    return new Foo(arg);
} //要掉用这个函数，调用者在调用后必须释放掉它

void use_factory(T arg){
    Foo *p = factory(arg);
    delete p;
}//如果没有释放掉它，p是指向factory分配的内存的唯一指针，一旦use_factory返回，程序就没有办法在释放这快内存了
```

>使用new和delete管理动态内存存在的三个问题：   
>1.忘记delete内存，忘记释放动态内存，会导致人门常说的“内存泄漏”问题，因为这种内存永远不会归还自由空间了，查找内存泄漏错误是非常困难的，因为通常应用程序运行很长时间后，内存耗尽时，才能检测到    
>2.使用已经释放掉的对象，通过在释放内存后将指针置为空，有时可以检测出这种错误    
>3.同一块内存释放两次：两个内置类型的指针指向同一块自由空间分配的内存，在对一个指针进行delete之后，其指向的内存也会被释放，若再对第二个指针进行delete，会造成自由空间破坏

>**坚持使用智能指针**

** delete之后重置指针值 **

- 即使delete了某个内置类型的指针(也就是说释放了对应的内存空间)，虽然指针已经无效，但是其仍然保留这释放空间的对应地址，变成了空悬指针，也就是说我们要保留指针，可以将其置为空指针

In [1]:
%%writefile ../../Code/C++PrimerCode/chapter12/7.cpp
/*
 * This code is writed by htfeng.
 *
 * "Copyright (c) 2017 by Objectwrite."
 * Date: 2017-09-20
 * Time: 20:00pm
 *
 *  The code is the answer to exercise 7 of the elevnth chapter about the book "C++ Primer, Fifth Edition".
 *
 * If you have any question,please contact me.
 *
 * Email:1054708869@qq.com
*/
#include <iostream>    
#include <string>    
#include <vector>  
#include<memory>  
#include<list>  
using namespace std;    
  
//返回一个动态分配的vector,看第六题的意思是不用只能指针，那么将类型改为vector<int> *就好了  
shared_ptr<vector<int>> vector_i()  
{  
      
    shared_ptr<vector<int>> _ptr(new vector<int>);  
    return _ptr;  
}  
  
//给vector赋值  
void vector_j(shared_ptr<vector<int>> _ptr)  
{  
    int int_val;  
    while (cin>>int_val && int_val != 0)  
    {  
        _ptr->push_back(int_val);  
    }  
}  
  
//打印vector的值  
void vector_k(shared_ptr<vector<int>> _ptr)  
{  
    for (size_t i = 0; i < (*_ptr).size(); ++i)  
    {  
        cout<<(*_ptr)[i]<<endl;  
    }  
}  
  
int main(int argc,char **argv)    
{  
    shared_ptr<vector<int>> my_ptr = vector_i();  
    vector_j(my_ptr);  
    vector_k(my_ptr);  
    return 0;  
}    

Writing ../../Code/C++PrimerCode/chapter12/7.cpp


### shared_ptr和new结合使用

- 可以使用new返回的指针来初始化智能指针

```C++
shared_ptr<double> pl;
shared_ptr<int> p2(new int(42));  //p2指向一个值为42的int

shared_ptr<int> p1 = new int(1024); //错误，必须使用直接初始化形式
```

- shared_ptr<T> p(q): p管理内置指针q所指的对象;q必须指向new分配的内存

** 不能混合使用不同指针和智能指针 **

** 也不要使用get初始化另一个智能指针或为指针指针赋值 **

### 智能指针和异常 

- 使用内置指针管理内存时，如果在new和delete之间发生了异常，内存将不会被释放

```C++
void f(){
    int *ip = new int(42);
    //code  出现异常
    delete ip;
}
```

** 智能指针和哑类 **

** 使用我们自己的释放操作 **

### unique_ptr

- 与shared_ptr不同，某个时刻智能有一个unique_ptr指向一个给定的对象，当unique_ptr被销毁时，他所值的对象也被销毁
- 与shared_ptr不同的是，没有类似make_shared的标准库函数返回一个unique_ptr
- 当我们定义unique_ptr时，需要将其绑定到一个new返回的指针上

- 初始化unique_ptr

```C++
unique_ptr<double> p1;
unique_ptr<int> p2(new int(42));
```

- 不支持普通的拷贝和赋值

使用release和reset将指针的所有权从一个unique_ptr转移给另一个unique：

```C++
//将所有权从p1转移给p2
unique_ptr<string> p2(p1.release());  //release将p2置为空
unique_ptr<string> p3(new string("text"));
//将所有权从p3转移给p2
p2.reset(p3.release()); //reset释放了p2原来指向的内存
```

如果我们不用另一个智能指针保存release返回的指针，我们的程序就要负责资源的释放

```C++
p2.release();  //错误，p2不会释放内存，而且我们丢了指针

auto p = p2.release();  //正确，但是要记得delete(p)
```

** 传递unique_ptr参数和返回unique_ptr **

- 不能拷贝unique_ptr的规则有一个例外，可以拷贝或赋值一个将要被销毁的unique_ptr，最常见的例子是从函数返回一个unique_ptr

```C++
unique<int> clone(int p){
    return unique<int>(new int(p));
}
```
- 还可以返回一个局部对象的拷贝

```C++
unique<int> clone(int p){
    unique<int> ret(new int(p));
    return ret;
}
```

** 向unique_ptr传递删除器 **

- unique_ptr<objT, delT> p(new objT, fun);

### weak_ptr

- weak_ptr是一种不控制所指向对象生存期的智能指针，它指向一个由一个shared_ptr管理的对象，将一个weak_ptr绑定到一个shared_ptr不会改变的shared_ptr的引用计数
- 创建一个weak_pr时，要用一个shared_ptr来初始化它

```C++
auto p = make_shared<int>(42);
weak_ptr<int> w(p);  //w弱共享p，p引用计数不变
```

- 由于是弱共享，创建w不会改变p的引用计数，w指向的对象有可能被释放掉
- 由于对象肯呢个不存在，我们不能直接访问对象，必须调用lock，检查weak_ptr指向的对象是否存在。如果存在，lock返回一个指向共享对象的shared_ptr,与任何其他的shared_ptr类似，只要shared_ptr存在，他所指向的底层对象就会一直存在

```C++
if(shared_ptr<int> np = wp.lock()){
    //code
}
```

- 这段代码，只有当lock调用返回true时，才会进入if，在if中，使用np访问共享对象是安全的

** 检查指针类 **

In [1]:
%%writefile ../../Code/C++PrimerCode/chapter12/20.cpp
/*
 * This code is writed by htfeng.
 *
 * "Copyright (c) 2017 by Objectwrite."
 * Date: 2017-09-22
 * Time: 16:40pm
 *
 *  The code is the answer to exercise 20 of the twelfth chapter about the book "C++ Primer, Fifth Edition".
 *
 * If you have any question,please contact me.
 *
 * Email:1054708869@qq.com
*/
#include<chapter12.h>
int main(int argc, char**argv)    
{   
    ifstream in("1.txt");  
    string s;  
    StrBlob blob;  
    while (getline(in,s))  
    {  
        blob.pushback(s);  
    }  
    for (StrBlobPtr pbeg(blob.begin()), pend(blob.end()); pbeg != pend;  
        pbeg.incr())  
        cout << pbeg.deref() << std::endl;  
    return 0;  
}   

Writing ../../Code/C++PrimerCode/chapter12/20.cpp


## 动态数组

- new和delete运算符一次分配/释放一个对象，但某些应用需要一次为很多对象分配内存的功能。例如，vector和string都是在连续内存中保存他们的元素，因此，当容器需要重新分配内存时，必须一次性为很多元素分配内存
- C++语言和标准库提供了两种一次分配一个对象数组的方法。c++定义了另一种new表达式语法，可以分配并初始化一个对象数组
- allocator类

### new和数组

```C++
typedef int arrT[42];  //arrT表示42个int的数组类型
int *p = new arrT;   //分配一个42个int的数组；p指向第一个int
```

** 分配一个数组会得到一个元素类型的指针  **

- 我们所说的数组并不是数组类型

** 初始化动态分配对象的数组 **

```C++
int *pia = new int[10];            //10个为初始化的int
int *pia2  = new int[10]();        //10个值初始化的0的int
string *psa = new string[10];      //10个空string
string *psa2 = new string[10]();   //10个空string

int *pia3 = new int[10]{0,1,2,3,4,5,6,7,8,9};
string *psa3 = new string[10]{"a", "an", "the", string(3, 'X')};
```

** 动态分配一个空数组是合法的 **

** 释放动态数组 **

```C++
delete p;     //p必须指向一个动态分配的对象或为空
delete [] p;  //p必须指向一个动态分配的数组或为空
```

** 智能指针和动态数组 **

- 标准库提供了一个可以管理new分配的数组的unique_ptr版本，为了unique_ptr管理动态数组，我们必须在对象类型后面跟一对空方括号。

```C++
unique_ptr<int[]> up(new int[10]);
up.release(); //自动用delete[]销毁指针
```

- 与之前的unique_ptr的操作不同，指向数组的unique_ptr不能使用点或者箭头成员运算符，要使用小标
- 与unique_ptr不同，shared_ptr不直接支持管理动态数组，如果希望使用shared_ptr管理动态数组，必须提供自己定义的删除器

```C++
shared_ptr<int> sp(new int[10], [](int *p){delete [] p;});
sp.reset();   //使用我们提供的lambda释放数组，它使用delelte[]
```

- shared_ptr未定义小标运算符，为了访问数组元素，必须get获取一个内置指针，然后用它来访问数组。

```C++
for(size_t i = 0; i != 10; ++i)
    *(sp.get() + i) = i;   //使用get获取一个内置指针
```

In [2]:
%%writefile ../../Code/C++PrimerCode/chapter12/24.cpp
/*
 * This code is writed by htfeng.
 *
 * "Copyright (c) 2017 by Objectwrite."
 * Date: 2017-09-23
 * Time: 16:26pm
 *
 *  The code is the answer to exercise 24 of the twelfth chapter about the book "C++ Primer, Fifth Edition".
 *
 * If you have any question,please contact me.
 *
 * Email:1054708869@qq.com
*/
#include <iostream>
#include <string>
#include <vector>
#include <memory>
#include<algorithm>  
#include <string.h>

using namespace std;

int main(int argc, char **argv){
	string str;
	size_t i = 0;
	unique_ptr<string[]> ustr(new string[2]);
	while(cin >> str && str != "end"){
		ustr[i] = str;
		++i;
	}
	
	for(size_t j = 0; j != i; ++j){
		cout << ustr[j] << endl;
	}
	ustr.release();
	return 0;
}

Overwriting ../../Code/C++PrimerCode/chapter12/24.cpp


### allocator类

- 有时后我们为动态数据分配了n个对象，但是现实是只要用少量的就够了，这样我们创建了一些永远也用不到的对象
- 而且对于用到的对象，在初始化之后立即赋值，相当于被赋值了两次

- 标准库allocator类定义在memory头文件中，它帮助我们将内存分配和对象构造分离开来
- 当一个allocator对象分配内存时，他会根据给定的对象类型确定恰当的内存大小或对其方式

```C++
allocator<string> alloc;           //可以分配string的allocator对象
auto const p = alloc.allocate(n);  //分配n个未初始化的string
```


- 这个allocate调用n个string分配了内存

** allocator分配未构造的内存 **

- allocator分配的内存是未构造的，我们按需在此内存中构造对象
- 在新标准库中，construct成员函数接受一个指针和零个或者多个额外参数，在给定位置构造一个元素


```C++
auto q = p;  //q指向最后**构造的元素**之后的位置
alloc.construct(q++);  //*q为空字符串
alloc.construct(q++， 10, ‘c');  //*q为ccccccccc
alloc.construct(q++, "hi");  //*q为hi
```

- 使用完对象后，必须对每个构造的元素调用destory来销毁它们

```C++
while(q != p)
    alloc.destory(--q);
```

- 调用deallocate来释放内存

```C++
alloc.deallocate(p,n);
```

** 拷贝和填充为初始化内存的算法 **

- 标准库还为allocator类定义了两个伴随算法，可以在未初始化内存中创建对象

- uninitialized_copy(b, e, b2)
- uninitialized_copy_n(b, n, b2)
- uninitialized_fill(b, e, t)
- uninitialized_fill_n(b, n, t)

```C++
auto p = alloc.allocate(vi.size() * 2);  
auto q = uninitialized_copy(vi.begin(), vi.end(), p); //通过拷贝vi中的元素来构造从p开始元素
uninitialized_fill_n(q, vi.size(), 42);  //将剩余元素初始化42
```

In [4]:
%%writefile ../../Code/C++PrimerCode/chapter12/26.cpp
/*
 * This code is writed by htfeng.
 *
 * "Copyright (c) 2017 by Objectwrite."
 * Date: 2017-09-23
 * Time: 18:46pm
 *
 *  The code is the answer to exercise 26 of the twelfth chapter about the book "C++ Primer, Fifth Edition".
 *
 * If you have any question,please contact me.
 *
 * Email:1054708869@qq.com
*/
#include <iostream>
#include <string>
#include <vector>
#include <memory>
#include <algorithm> 

using namespace std;

int main(int argc, char **argv){
	size_t i = 0, n = 10;
	allocator<string> alloc;
	auto p = alloc.allocate(n);
	string s;
	auto q = p;
	
	cout << "input a string:";
	while(cin >> s && s != "end" && i <= n){
		alloc.construct(q++, s);
		i++;
		cout << "input a string:";
	}
	
	while(q != p)
		alloc.destroy(--q);

	alloc.deallocate(p, n);

	return 0;
} 

Overwriting ../../Code/C++PrimerCode/chapter12/26.cpp


## 使用标准库：文本查询程序